In [ ]:
from utils import SPECIAL_TOKENS
import os
import json
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

project_root = os.path.abspath("..")
DATASET_PATH = os.path.join(project_root, "datasets/processed_dataset.csv")

df = pd.read_csv(DATASET_PATH)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=SEED)

dataset = {
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
}

def tokenized(example):
    return tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

BEST_MODELS_PATH = "best_models.json"
F1_TOLERANCE = 1e-3

def load_best_models():
    if not os.path.exists(BEST_MODELS_PATH):
        return {}
    with open(BEST_MODELS_PATH, "r") as f:
        return json.load(f)

def update_best_model(model_name, f1_score, precision, recall, accuracy):
    best_models = load_best_models()
    current_best = best_models.get(model_name, {"f1": 0.0, "precision": 0.0})

    better_f1 = f1_score - F1_TOLERANCE > current_best["f1"]
    similar_f1 = abs(f1_score - current_best["f1"]) <= F1_TOLERANCE
    better_precision = precision > current_best["precision"]

    if better_f1 or (similar_f1 and better_precision):
        print(f"🎯 New best for {model_name}!")
        best_models[model_name] = {
            "f1": f1_score,
            "precision": precision,
            "recall": recall,
            "accuracy": accuracy,
        }
        with open(BEST_MODELS_PATH, "w") as f:
            json.dump(best_models, f, indent=2)
        return True
    else:
        print(f"🧪 {model_name} did not improve:")
        print(f"F1: {f1_score:.4f} (best: {current_best['f1']:.4f}) | Precision: {precision:.4f} (best: {current_best['precision']:.4f})")
        return False

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

model_id = "SkolkovoInstitute/roberta_toxicity_classifier"
model_tag = model_id.replace("/", "-").replace("_", "-")
model_name = f"{model_tag}-tokenized"
print(f"\n🚀 Training model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens({"additional_special_tokens": SPECIAL_TOKENS})

tokenized_dataset = {
    k: v.map(tokenized, batched=True) for k, v in dataset.items()
}
for split in tokenized_dataset:
    tokenized_dataset[split].set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "label"]
    )

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
model.resize_token_embeddings(len(tokenizer))
model.to(device)

training_args = TrainingArguments(
    output_dir=f"./results_{model_tag}",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_dir=f"./logs_{model_tag}",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
    optim="adamw_torch",
    adam_epsilon=1e-8,
    lr_scheduler_type="linear",
    seed=SEED,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
eval_result = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

print(eval_result["eval_f1"], eval_result["eval_precision"])

if update_best_model(
    model_name=model_name,
    f1_score=eval_result["eval_f1"],
    precision=eval_result["eval_precision"],
    recall=eval_result["eval_recall"],
    accuracy=eval_result["eval_accuracy"]
):
    trainer.save_model(f"models/best-{model_name}")


Using device: cuda

🚀 Training model: SkolkovoInstitute-roberta-toxicity-classifier-tokenized


Map:   0%|          | 0/8404 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1051 [00:00<?, ? examples/s]

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/tmp/ipykernel_3362/4163243425.py:

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.494411,0.740000,0.708934,0.873890,0.782816
2,0.684500,0.409968,0.797143,0.774295,0.877442,0.822648
3,0.684500,0.385154,0.819048,0.813445,0.859680,0.835924
4,0.331500,0.412231,0.826667,0.844485,0.829485,0.836918
5,0.331500,0.508944,0.835238,0.872137,0.811723,0.840846
6,0.213000,0.540076,0.837143,0.852518,0.841918,0.847185
7,0.213000,0.616542,0.835238,0.846975,0.845471,0.846222


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or